In [ ]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from cdei_helpers.fairness_measures import *
from scipy.optimize import minimize

import plotly.express as px
import plotly.graph_objs as go
from cdei_helpers.plot import group_box_plots, group_roc_curves

from copy import deepcopy

In [ ]:
def train_model(
    x,
    y,
    x_control,
    loss_function,
    apply_fairness_constraints,
    apply_accuracy_constraint,
    sep_constraint,
    sensitive_attrs,
    sensitive_attrs_to_cov_thresh,
    gamma=None,
):

    """
    Function that trains the model subject to various fairness constraints.
    If no constraints are given, then simply trains an unaltered classifier.
    Example usage in: "synthetic_data_demo/decision_boundary_demo.py"
    ----
    Inputs:
    X: (n) x (d+1) numpy array -- n = number of examples, d = number of features, one feature is the intercept
    y: 1-d numpy array (n entries)
    x_control: dictionary of the type {"s": [...]}, key "s" is the sensitive feature name, and the value is a 1-d list with n elements holding the sensitive feature values
    loss_function: the loss function that we want to optimize -- for now we have implementation of logistic loss, but other functions like hinge loss can also be added
    apply_fairness_constraints: optimize accuracy subject to fairness constraint (0/1 values)
    apply_accuracy_constraint: optimize fairness subject to accuracy constraint (0/1 values)
    sep_constraint: apply the fine grained accuracy constraint
        for details, see Section 3.3 of arxiv.org/abs/1507.05259v3
        For examples on how to apply these constraints, see "synthetic_data_demo/decision_boundary_demo.py"
    Note: both apply_fairness_constraints and apply_accuracy_constraint cannot be 1 at the same time
    sensitive_attrs: ["s1", "s2", ...], list of sensitive features for which to apply fairness constraint, all of these sensitive features should have a corresponding array in x_control
    sensitive_attrs_to_cov_thresh: the covariance threshold that the classifier should achieve (this is only needed when apply_fairness_constraints=1, not needed for the other two constraints)
    gamma: controls the loss in accuracy we are willing to incur when using apply_accuracy_constraint and sep_constraint
    ----
    Outputs:
    w: the learned weight vector for the classifier
    """

    assert (
        apply_accuracy_constraint == 1 and apply_fairness_constraints == 1
    ) == False  # both constraints cannot be applied at the same time

    max_iter = (
        100000  # maximum number of iterations for the minimization algorithm
    )

    if apply_fairness_constraints == 0:
        constraints = []
    else:
        constraints = get_constraint_list_cov(
            x, y, x_control, sensitive_attrs, sensitive_attrs_to_cov_thresh
        )

    if (
        apply_accuracy_constraint == 0
    ):  # its not the reverse problem, just train w with cross cov constraints

        f_args = (x, y)
        w = minimize(
            fun=loss_function,
            x0=np.random.rand(x.shape[1],),
            args=f_args,
            method="SLSQP",
            options={"maxiter": max_iter},
            constraints=constraints,
        )

    else:

        # train on just the loss function
        w = minimize(
            fun=loss_function,
            x0=np.random.rand(x.shape[1],),
            args=(x, y),
            method="SLSQP",
            options={"maxiter": max_iter},
            constraints=[],
        )

        old_w = deepcopy(w.x)

        def constraint_gamma_all(w, x, y, initial_loss_arr):

            gamma_arr = np.ones_like(y) * gamma  # set gamma for everyone
            new_loss = loss_function(w, x, y)
            old_loss = sum(initial_loss_arr)
            return ((1.0 + gamma) * old_loss) - new_loss

        def constraint_protected_people(
            w, x, y
        ):  # dont confuse the protected here with the sensitive feature protected/non-protected values -- protected here means that these points should not be misclassified to negative class
            return np.dot(
                w, x.T
            )  # if this is positive, the constraint is satisfied

        def constraint_unprotected_people(w, ind, old_loss, x, y):

            new_loss = loss_function(w, np.array([x]), np.array(y))
            return ((1.0 + gamma) * old_loss) - new_loss

        constraints = []
        predicted_labels = np.sign(np.dot(w.x, x.T))
        unconstrained_loss_arr = loss_function(w.x, x, y, return_arr=True)

        if sep_constraint == True:  # separate gemma for different people
            for i in range(0, len(predicted_labels)):
                if (
                    predicted_labels[i] == 1.0
                    and x_control[sensitive_attrs[0]][i] == 1.0
                ):  # for now we are assuming just one sensitive attr for reverse constraint, later, extend the code to take into account multiple sensitive attrs
                    c = {
                        "type": "ineq",
                        "fun": constraint_protected_people,
                        "args": (x[i], y[i]),
                    }  # this constraint makes sure that these people stay in the positive class even in the modified classifier
                    constraints.append(c)
                else:
                    c = {
                        "type": "ineq",
                        "fun": constraint_unprotected_people,
                        "args": (i, unconstrained_loss_arr[i], x[i], y[i]),
                    }
                    constraints.append(c)
        else:  # same gamma for everyone
            c = {
                "type": "ineq",
                "fun": constraint_gamma_all,
                "args": (x, y, unconstrained_loss_arr),
            }
            constraints.append(c)

        def cross_cov_abs_optm_func(weight_vec, x_in, x_control_in_arr):
            cross_cov = (
                x_control_in_arr - np.mean(x_control_in_arr)
            ) * np.dot(weight_vec, x_in.T)
            return float(abs(sum(cross_cov))) / float(x_in.shape[0])

        w = minimize(
            fun=cross_cov_abs_optm_func,
            x0=old_w,
            args=(x, x_control[sensitive_attrs[0]]),
            method="SLSQP",
            options={"maxiter": 100000},
            constraints=constraints,
        )

    try:
        assert w.success == True
    except:
        print(
            "Optimization problem did not converge.. Check the solution returned by the optimizer."
        )
        print("Returned solution is:")
        print(w)
        raise

    return w.x


def get_scores(w, x):
    scores = 1.0 / (1 + np.exp(-np.dot(x[: w.shape[0]], w)))

    return scores


def add_intercept(x):
    """ Add intercept to the data before linear classification """
    m, n = x.shape
    intercept = np.ones(m).reshape(m, 1)  # the constant b
    return np.concatenate((intercept, x), axis=1)


def logistic_loss(w, X, y, return_arr=None):
    """Computes the logistic loss.
    This function is used from scikit-learn source code
    Parameters
    ----------
    w : ndarray, shape (n_features,) or (n_features + 1,)
        Coefficient vector.
    X : {array-like, sparse matrix}, shape (n_samples, n_features)
        Training data.
    y : ndarray, shape (n_samples,)
        Array of labels.
    """
    yz = y * np.dot(X, w)
    # Logistic loss is the negative of the log of the logistic function.
    if return_arr == True:
        out = -(log_logistic(yz))
    else:
        out = -np.sum(log_logistic(yz))
    return out


def log_logistic(X):
    """ This function is used from scikit-learn source code. Source link below """

    """Compute the log of the logistic function, ``log(1 / (1 + e ** -x))``.
    This implementation is numerically stable because it splits positive and
    negative values::
        -log(1 + exp(-x_i))     if x_i > 0
        x_i - log(1 + exp(x_i)) if x_i <= 0
    Parameters
    ----------
    X: array-like, shape (M, N)
        Argument to the logistic function
    Returns
    -------
    out: array, shape (M, N)
        Log of the logistic function evaluated at every point in x
    Notes
    -----
    Source code at:
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/extmath.py
    -----
    See the blog post describing this implementation:
    http://fa.bianp.net/blog/2013/numerical-optimizers-for-logistic-regression/
    """
    if X.ndim > 1:
        raise Exception("Array of samples cannot be more than 1-D!")
    out = np.empty_like(X)  # same dimensions and data types

    idx = X > 0
    out[idx] = -np.log(1.0 + np.exp(-X[idx]))
    out[~idx] = X[~idx] - np.log(1.0 + np.exp(X[~idx]))
    return out


def get_constraint_list_cov(
    x_train,
    y_train,
    x_control_train,
    sensitive_attrs,
    sensitive_attrs_to_cov_thresh,
):
    """
    get the list of constraints to be fed to the minimizer
    """

    constraints = []
    for attr in sensitive_attrs:

        attr_arr = x_control_train[attr]
        attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)

        if index_dict is None:  # binary attribute
            thresh = sensitive_attrs_to_cov_thresh[attr]
            c = {
                "type": "ineq",
                "fun": test_sensitive_attr_constraint_cov,
                "args": (
                    x_train,
                    y_train,
                    attr_arr_transformed,
                    thresh,
                    False,
                ),
            }
            constraints.append(c)
        else:  # otherwise, its a categorical attribute, so we need to set the cov thresh for each value separately
            for attr_val, ind in index_dict.items():
                attr_name = attr_val
                thresh = sensitive_attrs_to_cov_thresh[attr][attr_name]

                t = attr_arr_transformed[:, ind]
                c = {
                    "type": "ineq",
                    "fun": test_sensitive_attr_constraint_cov,
                    "args": (x_train, y_train, t, thresh, False),
                }
                constraints.append(c)

    return constraints


def get_one_hot_encoding(in_arr):
    """
        input: 1-D arr with int vals -- if not int vals, will raise an error
        output: m (ndarray): one-hot encoded matrix
                d (dict): also returns a dictionary original_val -> column in encoded matrix
    """

    for k in in_arr:
        if (
            str(type(k)) != "<type 'numpy.float64'>"
            and type(k) != int
            and type(k) != np.int64
        ):
            print(str(type(k)))
            print("************* ERROR: Input arr does not have integer types")
            return None

    in_arr = np.array(in_arr, dtype=int)
    assert len(in_arr.shape) == 1  # no column, means it was a 1-D arr
    attr_vals_uniq_sorted = sorted(list(set(in_arr)))
    num_uniq_vals = len(attr_vals_uniq_sorted)
    if (num_uniq_vals == 2) and (
        attr_vals_uniq_sorted[0] == 0 and attr_vals_uniq_sorted[1] == 1
    ):
        return in_arr, None

    index_dict = {}  # value to the column number
    for i in range(0, len(attr_vals_uniq_sorted)):
        val = attr_vals_uniq_sorted[i]
        index_dict[val] = i

    out_arr = []
    for i in range(0, len(in_arr)):
        tup = np.zeros(num_uniq_vals)
        val = in_arr[i]
        ind = index_dict[val]
        tup[ind] = 1  # set that value of tuple to 1
        out_arr.append(tup)

    return np.array(out_arr), index_dict


def test_sensitive_attr_constraint_cov(
    model, x_arr, y_arr_dist_boundary, x_control, thresh, verbose
):

    """
    The covariance is computed b/w the sensitive attr val and the distance from the boundary
    If the model is None, we assume that the y_arr_dist_boundary contains the distace from the decision boundary
    If the model is not None, we just compute a dot product or model and x_arr
    for the case of SVM, we pass the distace from bounday becase the intercept in internalized for the class
    and we have compute the distance using the project function
    this function will return -1 if the constraint specified by thresh parameter is not satifsified
    otherwise it will reutrn +1
    if the return value is >=0, then the constraint is satisfied
    """

    assert x_arr.shape[0] == x_control.shape[0]
    if (
        len(x_control.shape) > 1
    ):  # make sure we just have one column in the array
        assert x_control.shape[1] == 1

    arr = []
    if model is None:
        arr = y_arr_dist_boundary  # simply the output labels
    else:
        arr = np.dot(
            model, x_arr.T
        )  # the product with the weight vector -- the sign of this is the output label

    arr = np.array(arr, dtype=np.float64)
    cov = np.dot(x_control - np.mean(x_control), arr) / float(len(x_control))
    ans = thresh - abs(
        cov
    )  # will be <0 if the covariance is greater than thresh -- that is, the condition is not satisfied
    # ans = thresh - cov # will be <0 if the covariance is greater than thresh -- that is, the condition is not satisfied
    if verbose is True:
        print("Covariance is", cov)
        print("Diff is:", ans)
        print()
    return ans

## Load data

In [ ]:
train = pd.read_csv("/project/data/adult/processed/train-one-hot.csv").sample(
    2000
)
test = pd.read_csv("/project/data/adult/processed/test-one-hot.csv").sample(
    2000
)
val = pd.read_csv("/project/data/adult/processed/val-one-hot.csv").sample(6000)

In [ ]:
x_train = train.drop(["sex", "salary"], axis=1).values
y_train = train["salary"].values
y_train[y_train == 0] = -1
x_control_train = {"sex": train["sex"].values}
a_train = x_control_train["sex"]

x_test = test.drop(["sex", "salary"], axis=1).values
y_test = test["salary"].values
y_test[y_test == 0] = -1
x_control_test = {"sex": test["sex"].values}
a_test = x_control_test["sex"]

In [ ]:
apply_fairness_constraints = 1  # set this flag to one since we want to optimize accuracy subject to fairness constraints
apply_accuracy_constraint = 0
sep_constraint = 0
sensitive_attrs = ["sex"]
cov_factor = 0.1
sensitive_attrs_to_cov_thresh = {"sex": cov_factor}
loss_function = logistic_loss
x_train = add_intercept(
    x_train
)  # add intercept to X before applying the linear classifier
x_test = add_intercept(x_test)
gamma = None

## Set type of interevention

### Optimise for accuracy only, i.e., unconstrained

In [ ]:
apply_fairness_constraints = 0
apply_accuracy_constraint = 0
sep_constraint = 0
sensitive_attrs_to_cov_thresh = None
gamma = None

### Optimise for accuracy under fairness constraint

In [ ]:
apply_fairness_constraints = 1
apply_accuracy_constraint = 0
sep_constraint = 0
sensitive_attrs_to_cov_thresh = {
    "sex": 0.1
}  # controls covariance to protected attribute
gamma = None

### Optimise for fairness under accuracy constraint

In [ ]:
apply_fairness_constraints = 0
apply_accuracy_constraint = 1
sep_constraint = 0
sensitive_attrs_to_cov_thresh = None
gamma = 0.3  # controls priority of fairness over accuracy

### Further parameters

In [ ]:
# sensitive attribute
sensitive_attrs = ["sex"]

# set loss function
loss_function = logistic_loss

# add intercept to X before applying the linear classifier
x_train = add_intercept(x_train)
x_test = add_intercept(x_test)

## Perform intervention

In [ ]:
%%time

w = train_model(
    x_train,
    y_train,
    x_control_train,
    loss_function,
    apply_fairness_constraints,
    apply_accuracy_constraint,
    sep_constraint,
    sensitive_attrs,
    sensitive_attrs_to_cov_thresh,
    gamma,
)

## Evaluate demographic parity and accuracy

In [ ]:
test_scores = get_scores(x_test.T, w)

In [ ]:
y_test[y_test == -1] = 0
acc = accuracy(test_scores, test.salary)
print("Accuracy after fairness intervention =", acc)

In [ ]:
dp_d = independence_d(test_scores, test.sex)
print("Demographic parity =", 1.0 - dp_d)

In [ ]:
go.Figure(
    data=[
        go.Bar(
            x=[sex],
            y=[test_scores[test.sex == sex].mean()],
            name="Male" if sex else "Female",
        )
        for sex in range(2)
    ]
)